# Video service

if you have errors running sample code reach out for help in #igiu-ai-learning
other questions #igiu-innovation-lab or #video-analysis-beta

Note video support is not GA and requires a specific version of oci & video client. reach out to ashish.ag.agarwal@oracle.com

## Import Libraries

In [1]:
import oci
import io,json,os
from oci.ai_vision.models import *
from oci.object_storage import ObjectStorageClient
from time import sleep

## Set input variables



In [2]:
#####
#make sure your sandbox.json file is setup for your environment. You might have to specify the full path depending on  your `cwd` 
# you can also try making your cwd ofr jupyter match your workspace python code: 
# vscopde menu -> Settings > Extensions > Jupyter > Notebook File Root
# change from ${fileDirname} to ${workspaceFolder}
#####

#SANDBOX_CONFIG_FILE = "~/work/code/python/workshop/sandbox.json"
SANDBOX_CONFIG_FILE = "sandbox.json"
FILE_TO_ANALYZE = "./vision/mall.mp4" #specify the filename of the test video

# Read the config file

In [3]:
scfg = None
# read the sandbox config 
with open(os.path.expanduser(SANDBOX_CONFIG_FILE), 'r') as f:
                scfg=  json.load(f)

oci_cfg = oci.config.from_file(os.path.expanduser(scfg["oci"]["configFile"]),scfg["oci"]["profile"])
bucket_cfg = scfg["bucket"]
namespace = bucket_cfg["namespace"]
bucketName =  bucket_cfg["bucketName"]
filename = os.path.basename(FILE_TO_ANALYZE)
prefix = bucket_cfg['prefix']
compartmentId =scfg["oci"]["compartment"] 

## Upload file  

This is an optional step. If the file is already uploaded, no need to upload it again.

In [ ]:
object_storage_client = ObjectStorageClient(oci_cfg)
print(f"Uploading file {FILE_TO_ANALYZE} ...")
object_storage_client.put_object(bucket_cfg["namespace"], 
                                bucket_cfg["bucketName"], 
                                f"{bucket_cfg['prefix']}/{os.path.basename(FILE_TO_ANALYZE)}", 
                                io.open(FILE_TO_ANALYZE,'rb'))
print("Upload completed !")

## Create AI service vision client

In [5]:
ai_service_vision_client = oci.ai_vision.AIServiceVisionClient(config=oci_cfg)

## Setup input feature for Offline video features
You can specify the features you want to call. In the below code we are calling all the features

In [6]:
video_object_detection_feature = VideoObjectDetectionFeature()
video_face_detection_feature = VideoFaceDetectionFeature()
video_label_detection_feature = VideoLabelDetectionFeature()
video_text_detection_feature = VideoTextDetectionFeature()
features = [video_text_detection_feature, video_face_detection_feature, video_label_detection_feature, video_object_detection_feature]

## Setup input location

This example assumes that your file ot analyze is already in teh bucket. You can either load it manually to the bucket using the oci console from your browser, or do it programatically. see vision notebook section on upload. 

In [7]:
object_location_1 = ObjectLocation()
object_location_1.namespace_name = namespace
object_location_1.bucket_name = bucketName
object_location_1.object_name =  f"{prefix}/{filename}"
object_locations = [object_location_1]
input_location = ObjectListInlineInputLocation()
input_location.object_locations = object_locations

## Setup output location

In [8]:
output_location = OutputLocation()
output_location.namespace_name = namespace
output_location.bucket_name = bucketName
output_location.prefix = prefix

## Create video job

In [ ]:
create_video_job_details = CreateVideoJobDetails()
create_video_job_details.features = features
create_video_job_details.compartment_id = compartmentId
create_video_job_details.output_location = output_location
create_video_job_details.input_location = input_location

res = ai_service_vision_client.create_video_job(create_video_job_details=create_video_job_details)

job_id = res.data.id
print(f"Job {job_id} is in {res.data.lifecycle_state} state.")


## Job in progress
The job progress is tracked till completion with an interval of 5 seconds.

In [ ]:
seconds = 0
while res.data.lifecycle_state == "IN_PROGRESS" or res.data.lifecycle_state == "ACCEPTED":
    print(f"Job {job_id} is IN_PROGRESS for {str(seconds)} seconds, progress: {res.data.percent_complete}")
    sleep(5)
    seconds += 5
    res = ai_service_vision_client.get_video_job(video_job_id=job_id)
    
print(f"Job {job_id} is in {res.data.lifecycle_state} state.")

## Get response json from object storage
The output can be found in the output location specified or it can be saved in ```output.json``` file by running the below cell

In [ ]:
object_storage_client = oci.object_storage.ObjectStorageClient(oci_cfg)
object_name = f"{prefix}/{job_id}/{prefix}/{filename}.json"

video_response = object_storage_client.get_object(namespace, bucketName, object_name)


print (video_response.data.text)